<a href="https://colab.research.google.com/github/ELGARNAOUI/earthengine-community/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import ee
import geemap

In [37]:
ee.Authenticate()
ee.Initialize(project="focus-hulling-364814", opt_url='https://earthengine-highvolume.googleapis.com')

In [38]:
country_name ='Morocco'                           # Country's name            # str
time_start = "202-01-01"                            # Start data YYYY-MM-DD     # str
time_end =  "2020-12-30"                              # End data YYYY-MM-DD       # str

In [39]:
def convert_to_monthly_sum(collection, start, end):
    start_date = ee.Date(start)
    end_date = ee.Date(end)

    adjusted_end = ee.Algorithms.If(
        ee.Number(end_date.get('day')).eq(1),
        end_date.advance(-1, 'day'),
        end_date
    )
    end_date = ee.Date(adjusted_end)

    months = ee.List.sequence(0, end_date.difference(start_date, 'month'))

    def map_month(m):
        m = ee.Number(m)
        start_m = start_date.advance(m, 'month')
        end_m = start_m.advance(1, 'month')

        filtered = collection.filterDate(start_m, end_m)
        monthly_image = filtered.sum() \
            .set('system:time_start', start_m.millis())

        first_image = ee.Image(filtered.first())
        monthly_image = monthly_image.copyProperties(first_image, first_image.propertyNames())

        return monthly_image

    return ee.ImageCollection.fromImages(months.map(map_month))


def convert_to_monthly_average(collection, start, end):
    start_date = ee.Date(start)
    end_date = ee.Date(end)

    adjusted_end = ee.Algorithms.If(
        ee.Number(end_date.get('day')).eq(1),
        end_date.advance(-1, 'day'),
        end_date
    )
    end_date = ee.Date(adjusted_end)

    months = ee.List.sequence(0, end_date.difference(start_date, 'month'))

    def map_month(m):
        m = ee.Number(m)
        start_m = start_date.advance(m, 'month')
        end_m = start_m.advance(1, 'month')

        filtered = collection.filterDate(start_m, end_m)
        monthly_image = filtered.mean() \
            .set('system:time_start', start_m.millis())

        first_image = ee.Image(filtered.first())
        monthly_image = monthly_image.copyProperties(first_image, first_image.propertyNames())

        return monthly_image

    return ee.ImageCollection.fromImages(months.map(map_month))

In [40]:
country = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
    .filter(ee.Filter.eq('country_na', country_name))
roi=country.geometry()

In [41]:
grace_55km = (
    ee.ImageCollection("NASA/GRACE/MASS_GRIDS_V04/MASCON_CRI")
    .select('lwe_thickness')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

In [42]:
precipitation = (
    ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06")
    .select("precipitation")
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

precipitation_monthly = convert_to_monthly_sum(precipitation, time_start, time_end)

In [43]:
air_temperature = (
    ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR")
    .select("temperature_2m")
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

air_temperature_monthly = convert_to_monthly_average(air_temperature, time_start, time_end)

In [44]:
land_temperature_monthly = (
    ee.ImageCollection("Oxford/MAP/LST_Day_5km_Monthly")
    .select('Mean')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

In [45]:
humidity = (
    ee.ImageCollection("UCSB-CHG/CHIRTS/DAILY")
    .select("relative_humidity")
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

humidity_monthly = convert_to_monthly_average(humidity, time_start, time_end)

In [46]:
evaporation = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('potential_evaporation_sum')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

evaporation_monthly = convert_to_monthly_average(evaporation, time_start, time_end)

In [47]:
runoff = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('runoff_sum')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

runoff_monthly = convert_to_monthly_sum(runoff, time_start, time_end)

In [48]:
ndvi = (
    ee.ImageCollection("MODIS/061/MOD13A2")
    .select('NDVI')
    .filterDate(time_start, time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

ndvi_monthly = convert_to_monthly_average(ndvi, time_start, time_end)

In [49]:
def join_collections(primary, secondary, band_name):
    join = ee.Join.inner()
    filter = ee.Filter.equals(leftField='date', rightField='date')
    joined = join.apply(primary, secondary, filter)

    return ee.ImageCollection(joined.map(lambda feature:
        ee.Image(feature.get('primary')).addBands(
            ee.Image(feature.get('secondary')).rename(band_name)
        )
    ))


joined1 = join_collections(grace_55km, precipitation_monthly, 'Precipitation')
joined2 = join_collections(joined1, air_temperature_monthly, 'Air temperature')
joined3 = join_collections(joined2, land_temperature_monthly, 'Land Surface temperature')
joined4 = join_collections(joined3, humidity_monthly, 'Humidity')
joined5 = join_collections(joined4, evaporation_monthly, 'Evaporation')
joined6 = join_collections(joined5, runoff_monthly, 'Runoff')
data_collection = join_collections(joined6, ndvi_monthly, 'NDVI')

In [50]:
def classify_grace10km(img):
    training = img.stratifiedSample(
        numPoints=100,
        classBand='lwe_thickness',
        region=roi,
        scale=10000
    )

    model = ee.Classifier.smileRandomForest(80).train(
        features=training,
        classProperty='lwe_thickness',
        inputProperties=img.bandNames()
    ).setOutputMode('REGRESSION')

    bands = img.bandNames().remove('lwe_thickness')

    result = img.select(bands).classify(model).rename('grace10km').toFloat()

    return result.copyProperties(img, img.propertyNames())

grace10km = data_collection.map(classify_grace10km)

In [52]:
# Create a new map with a satellite basemap
Map = geemap.Map(basemap="SATELLITE")

# -------------------------------
# Loop over the 55 km GRACE collection
# -------------------------------
count_55km = grace_55km.size().getInfo()
image_list_55km = grace_55km.toList(count_55km)

for i in range(count_55km):
    img = ee.Image(image_list_55km.get(i)).clip(roi)
    date = img.get('date').getInfo()

    stats = img.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        bestEffort=True
    ).getInfo()

    vis_55km = {
        'min': stats['lwe_thickness_min'],
        'max': stats['lwe_thickness_max'],
        # 5‐color diverging palette: dark blue → light blue → white → light red → dark red
        'palette': [
            '#2166ac',  # dark blue (strong positive change)
            '#67a9cf',  # light blue (moderate positive)
            '#f7f7f7',  # white     (no change)
            '#ef8a62',  # light red (moderate negative)
            '#b2182b'   # dark red  (strong negative change)
        ]
    }

    Map.addLayer(img, vis_55km, f"Grace 55km {date}")

# -------------------------------
# Loop over the 10 km GRACE collection
# -------------------------------
count_10km = grace10km.size().getInfo()
image_list_10km = grace10km.toList(count_10km)

for i in range(count_10km):
    img = ee.Image(image_list_10km.get(i)).clip(roi)
    date = img.get('date').getInfo()

    stats = img.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        bestEffort=True
    ).getInfo()

    vis_10km = {
        'min': stats['grace10km_min'],
        'max': stats['grace10km_max'],
        # Same 5‐color diverging palette for consistency
        'palette': [
            '#2166ac',  # dark blue (strong positive change)
            '#67a9cf',  # light blue (moderate positive)
            '#f7f7f7',  # white     (no change)
            '#ef8a62',  # light red (moderate negative)
            '#b2182b'   # dark red  (strong negative change)
        ]
    }

    Map.addLayer(img, vis_10km, f"Grace 10km {date}")

# Center the map on the region of interest at zoom level 5
Map.centerObject(roi, 5)

# Display the map
Map


EEException: Element.propertyNames: Parameter 'element' is required and may not be null.